In [11]:
import numpy as np
import pandas as pd

#### Results at end. Prior code builds up to resulting score_advertisers func at bottom.

In [12]:
ADVERTISERS = pd.read_json("lookup/advertiser_id/map.json", lines=False)

In [ ]:
# Assumes presence of data. Was not added to GitHub.
table = pd.read_json("./part-00000", lines=True)

In [14]:
# Choose relevant columns
click_counts = table[["advertiser_id", "c_cnt"]]

In [15]:
# Drop bidder rows
clean_ccnt = click_counts.dropna()

In [16]:
clean_ccnt.groupby("advertiser_id").agg({'c_cnt': [np.sum, np.size, np.mean]}).head()

c_cnt                  
                sum    size      mean
advertiser_id                        
534.0           0.0    77.0  0.000000
570.0           0.0   141.0  0.000000
832.0           0.0     8.0  0.000000
840.0           2.0  2516.0  0.000795
981.0           0.0     4.0  0.000000

In [18]:
for col in ADVERTISERS:
    print(col)

advertiser_id
advertiser_name


In [21]:
aggregated = pd.merge(clean_ccnt, ADVERTISERS).groupby("advertiser_name").agg({'c_cnt': np.mean})

In [22]:
aggregated.head()

,c_cnt
advertiser_name,
AVRIL,0.00000
AdWeb Ltd,0.02439
Alena,0.00000
Appnext,0.00000
Audible,1.00000


In [23]:
aggregated[(aggregated["c_cnt"] != 1) & (aggregated["c_cnt"] > 0)].mean()

c_cnt    0.011162
dtype: float64

In [24]:
aggregated[(aggregated["c_cnt"] > 0)].mean()

c_cnt    0.044123
dtype: float64

In [25]:
aggregated[(aggregated["c_cnt"] != 1)].mean()

c_cnt    0.002865
dtype: float64

In [26]:
aggregated.mean()

c_cnt    0.011611
dtype: float64

In [27]:
import math

def sigmoid(x):
    return 2*(1 / (1 + math.exp(-x)) - 0.5)

In [28]:
sigmoid(1)

0.4621171572600098

In [30]:
agg_norm = pd.merge(clean_ccnt, ADVERTISERS).groupby("advertiser_name").agg({'c_cnt': lambda x: sigmoid(np.mean(x))})

In [31]:
agg_norm.head(10)

,c_cnt
advertiser_name,
AVRIL,0.000000
AdWeb Ltd,0.012195
Alena,0.000000
Appnext,0.000000
Audible,0.462117
Australia tourism,0.000000
Avedura Investments LTD,0.000000
Beverly Hills MD,0.000000
Bidswitch RTB,0.000442


In [1]:
# Version of scoring advertisers that replaces advertiser IDs with their names
# Requires definition of ADVERTISERS dictionary that maps IDs to names
# ^^ not necessary. Version without this is located at bottom of notebook

def score_advertisers(table, score_func):
    click_counts = table[["advertiser_id", "c_cnt"]]
    clean_ccnt = click_counts.dropna()
    merged = pd.merge(clean_ccnt, ADVERTISERS)
    grouped = merged.groupby("advertiser_name")
    score_means = lambda x: score_func(np.mean(x))
    agg_scored = grouped.agg({'c_cnt': score_means})
    agg_scored = agg_scored.sort_values(by="c_cnt", ascending=False)
    return agg_scored

In [33]:
score_advertisers(table, sigmoid).head(10)

,c_cnt
advertiser_name,
Audible,0.462117
Viber - Clickers,0.032883
admixer RTB,0.024995
Facebook,0.023360
AdWeb Ltd,0.012195
"Rakuten, Inc",0.010869
Diet,0.008620
Weekly National,0.008620
Brain,0.005952


In [34]:
def sigmoid2(x):
    return 3*(1 / (1 + math.exp(-x)) - 0.5)

score_advertisers(table, sigmoid2).head(10)

,c_cnt
advertiser_name,
Audible,0.693176
Viber - Clickers,0.049324
admixer RTB,0.037492
Facebook,0.035040
AdWeb Ltd,0.018292
"Rakuten, Inc",0.016304
Diet,0.012931
Weekly National,0.012931
Brain,0.008928


In [35]:
def sigmoid3(x):
    return 4*(1 / (1 + math.exp(-x)) - 0.5)

score_advertisers(table, sigmoid3).head().sort_values(by="c_cnt", ascending=False)

,c_cnt
advertiser_name,
Audible,0.924234
Viber - Clickers,0.065766
admixer RTB,0.049990
Facebook,0.046720
AdWeb Ltd,0.024389


In [36]:
score_advertisers(table, lambda x: x).head().sort_values(by="c_cnt", ascending=False)

,c_cnt
advertiser_name,
Audible,1.000000
Viber - Clickers,0.065789
admixer RTB,0.050000
Facebook,0.046729
AdWeb Ltd,0.024390


In [2]:
# Cleaned function in order to find advertiser click-ratios
# Takes in score_func to adjust distribution of ratios for better presentation/visual analysis
# Automatically sorts values (descending)

def score_advertisers(table, score_func):
    click_counts = table[["advertiser_id", "c_cnt"]]
    clean_ccnt = click_counts.dropna()
    grouped = clean_ccnt.groupby("advertiser_id")
    score_means = lambda x: score_func(np.mean(x))
    agg_scored = grouped.agg({'c_cnt': score_means})
    agg_scored = agg_scored.sort_values(by="c_cnt", ascending=False)
    return agg_scored

In [42]:
score_advertisers(table, lambda x: x)

,c_cnt
advertiser_id,
5029.0,1.000000
4315.0,0.065789
3994.0,0.059524
3686.0,0.050000
4001.0,0.035714
4412.0,0.024390
4326.0,0.021739
5101.0,0.017241
4522.0,0.017241
